# Preliminary Notebook

## 1. Import and Install Dependencies

In [34]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib

Note: you may need to restart the kernel to use updated packages.


In [35]:
import cv2 # opencv
import numpy as np
import os # easier file path handling
from matplotlib import pyplot as plt # im.show for easy visualization
import time # to insert "sleep" in between frames
import mediapipe as mp # for accessing and reading from webcam

## 6. Preprocess Data and Create Labels and Features

In [36]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [37]:
# path for exported data (numpy arrays)
#DATA_PATH = os.path.join('MP_Data')
DATA_PATH = os.path.join('MP_Data_test')

# actions to detect
#actions = np.array(['hello', 'thanks', 'iloveyou'])
actions = np.array ( ['alligator', 'radio', 'moon', 'sleep', 'grandpa', 'tiger', 'pencil', 'sleepy', 'grandma', 'chocolate'])

# 30 videos of data
no_sequences = 30

# each video with 30 frames
sequence_length = 30

In [38]:
# create label map (dict, our .json file)
label_map = {label:num for num, label in enumerate(actions)}

In [39]:
sequences, labels = [], [] # sequences will be x data, labels will be y data
# loop over all actions (words)
for action in actions: 
    # loop over all sequences (videos)
    for sequence in range(no_sequences): 
        window = [] # represents all frames of particular sequence (video)
        # loop through each frame
        for frame_num in range(sequence_length): 
            # load up current frame (frame_num)
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res) # append to one video
        sequences.append(window) # append all videos to sequence (for a word)
        labels.append(label_map[action])

FileNotFoundError: [Errno 2] No such file or directory: 'MP_Data_test/alligator/0/0.npy'

## Feeding in preprocessed kaggle data

In [ ]:
#loading our preprocessed kaggle data
X = np.load('/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/data/X-data.npy')
y = np.load('/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/data/y-data.npy')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Feeding in our sample data

In [ ]:
#loading our own sample data
X_j = np.array(sequences) # shape: (90, 30, 1662)
y_j = to_categorical(labels).astype(int) # one-hot encoded labels (words)

In [ ]:
X_j_train, X_j_test, y_j_train, y_j_test = train_test_split(X_j, y_j, test_size=0.05)

In [ ]:
X_j_train.shape, X_train.shape

((85, 30, 1662), (315, 30, 1086))

In [ ]:
y_j_train.shape, y_train.shape

((85, 3), (315, 10))

**Problem: Number of signs changes the shape. Where do we change this in the model?**

## 7. Build and Train LSTM Neural Network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1086))) #changed to 1086 columns
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
# categorical_crossentropy must be used for multiclass classification model! 

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])
# advantage of using mediapipe holistic model is you don't need additional data generator to build up a pipeline of data. Training data fits all into memory.

Epoch 1/2000


ValueError: in user code:

    File "/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/Users/ronjaweiblen/Bootcamp/Capstone_Project_SignMeUp/.venv/lib/python3.9/site-packages/keras/engine/training.py", line 11

In [ ]:
# %load_ext tensorboard

In [ ]:
# to run it in the notebook: 
# %tensorboard --logdir=./Logs/train --port=8008

# to run it in terminal: python3 -m tensorboard --logdir=./Logs/train --port=8008
# then copy+paste this into your internet browser: localhost:8008

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-trai

## Make Predictions

In [ ]:
res = model.predict(X_test)

1/1 [==============================] - 0s 351ms/step


Some single example predictions :D

In [ ]:
actions[np.argmax(res[0])] # predictions

'hello'

In [ ]:
actions[np.argmax(y_test[0])] # actual

'hello'

## 9. Save Weights

In [ ]:
#model.save('first_model_whoop_whoop.h5')